# Algoritmo - Localização Bilhetagem (fora terminais BRT)
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Mesclagem por informações coincidentes e critério de interpolação_

# 1.0  Leitura das Bases

## 1.1 leitura da base - Bilhetagem (Tratamento Inicial)

In [ ]:
# Arquivos são do mês de novembro de 2018.
import pandas as pd

# Definindo tipos de variáveis com tamanhos menores
tipos_specification = {'id': 'int32', 'linha': 'int32'}

# Lista para armazenar os dataframes de cada grupo de 10 dias
grupos_dias = []

# Loop para ler os arquivos CSV e concatená-los em grupos de 10 dias
for i in range(1, 31, 10):  # Início, fim (não inclusivo), passo
    dias = [pd.read_csv(f'2018-11-{str(j).zfill(2)}.csv', usecols=['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'sentido_viagem', 'dia', 'momento'], dtype=tipos_specification, sep=',') for j in range(i, i + 10)]
    grupo = pd.concat(dias)
    grupos_dias.append(grupo)

In [ ]:
# Concatenando os grupos de 10 dias
mes_tudo = pd.concat(grupos_dias)

In [ ]:
# BUSCAR DADOS NULOS
mes_tudo.info()

In [ ]:
# Momentos em ordem do df mes_c_R!
# mes_tudo.sort_values('momento', inplace = True) # Mostrando DF em ORDEM!

In [ ]:
# Memoria liberou 0.2 gb
mes_tudo.info()

In [ ]:
# Juntando colunas de interesse (dia, momento) em uma só:
mes_tudo["momento"] = mes_tudo['dia'].astype(str) + ' ' + mes_tudo['momento'].astype(str)

In [ ]:
# REMOVENDO COLUNAS DESNECESSÁRIAS!
mes1 = mes_tudo.drop(['dia'], axis=1)

In [ ]:
# Excluindo explicitamente variavel da bilhetagem não mais necessária 
del mes_tudo

In [ ]:
mes1.info()

In [ ]:
mes1["momento"] = pd.to_datetime(mes1["momento"])

In [ ]:
# Removendo dados nulos na coluna prefixo_carro
mes_ = mes1.dropna(subset = ['prefixo_carro'])

In [ ]:
# Excluindo explicitamente variavel da bilhetagem não mais necessária 
del mes1

In [ ]:
# Removendo identificação de veiculos com caracteris da coluna de prefixo_carro
mes = mes_[pd.to_numeric(mes_['prefixo_carro'], errors='coerce').notnull()]

In [ ]:
# Excluindo explicitamente variavel da bilhetagem não mais necessária 
del mes_

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
mes['prefixo_carro'] = mes['prefixo_carro'].astype('int32')

In [ ]:
mes.info()

In [ ]:
# Última versão do df MES
mes

In [ ]:
import pandas as pd 

## 1.2 leitura da base - Dicionário

In [ ]:
# df', abaixo:
d1 = pd.read_csv('veiculos2018.csv', sep= ';')

In [ ]:
# Mostrando Df
display(d1)

In [ ]:
d1.info()

In [ ]:
# Removenfdo identificação de veiculos com caracters da coluna de cod_veiculo
d01 = d1[pd.to_numeric(d1['cod_veiculo'], errors='coerce').notnull()]

In [ ]:
d01

In [ ]:
d01.info()

In [ ]:
# Trasnformando coluna de prefixo carro em inteiro
d01['cod_veiculo'] = d01['cod_veiculo'].astype('int32')
d01['id_veiculo'] = d01['id_veiculo'].astype('int32')

In [ ]:
d01[d01.cod_veiculo ==36981]

In [ ]:
d01['cod_veiculo'].sort_index()

In [ ]:
d01.isnull().sum()

## 1.3 leitura do base - GPS (Tratamento inicial)

In [ ]:
import pandas as pd

# Definindo o número de linhas que você deseja ler
numero_de_linhas_para_ler = 45000000  # Substitua pelo número desejado
dtype_specification = {'longitude': 'float32', 'latitude': 'float32', 'vehicle_vehicleid': 'int32'}

# Usando o pandas para ler o CSV, limitando o número de linhas com a opção nrows
mes_ate_45_milhoes = pd.read_csv('Paint112018.csv',usecols=['latitude', 'longitude', 'metrictimestamp', 'vehicle_vehicleid'], dtype=dtype_specification, nrows=numero_de_linhas_para_ler)

In [ ]:
# Lendo restante do arquivo
mes_resto = pd.read_csv('Paint112018.csv', usecols=['latitude', 'longitude','metrictimestamp', 'vehicle_vehicleid'],dtype=dtype_specification, skiprows=range(1, numero_de_linhas_para_ler + 1))

In [ ]:
# Juntando em um mesmo df
GPS_I_H =  pd.concat([mes_ate_45_milhoes, mes_resto]) 

In [ ]:
# Deletando explicitamente variaveis com dados do GPS separados
del mes_ate_45_milhoes
del mes_resto

In [ ]:
display(GPS_I_H.iloc[275])

In [ ]:
GPS_I_H.info()

In [ ]:
# FORMATANDO metrictimestamp do (GPS)
GPS_I_H['momento_gps'] = pd.to_datetime(GPS_I_H['metrictimestamp'],format= '%Y%m%d%H%M%S')

In [ ]:
GPS_I_H.drop("metrictimestamp", axis= 1, inplace= True)

In [ ]:
# Renomear coluna de momento de vehicle_vehicleid para id_veiculo:
GPS_I_H.columns=['latitude', 'longitude','id_veiculo', 'momento_gps']

In [ ]:
# Colocando valores em ordem
GPS_I_H.sort_values('momento_gps', inplace= True)

In [ ]:
GPS_I_H.info()

## 1.3.1 Separando base do GPS Bruto caso queria localizar somente um dia especifico
''' Em casos que a integração será dos mes completo basta comentar esse topico e substituir o nome desse arquivo final "dia_especifico_gps_bruto" na função "identificação_momentos" pelo nome da base completa do GPS no topico 3.1 '''

In [ ]:
# Filtrando dia 01
# dia_especifico_gps_bruto_ = GPS_I_H[(GPS_I_H.momento_gps<='2018-11-01 23:59:59')]

In [ ]:
# Colocando df em ordem na coluna momento
# dia_especifico_gps_bruto = dia_especifico_gps_bruto_.sort_values('momento_gps')

In [ ]:
# Reordenar a colunas
# dia_especifico_gps_bruto = dia_especifico_gps_bruto.reset_index()

In [ ]:
# dia_especifico_gps_bruto

In [ ]:
# Deletando explicitamente dados do GPS 
# del GPS_I_H

# 2.0 Junção da bilhetagem com o dicionário 

In [ ]:
# Acrescentando dicionário
mes_d1_T = mes.merge(d01, left_on= 'prefixo_carro', right_on='cod_veiculo')

In [ ]:
mes_d1 = mes_d1_T

In [ ]:
# Deletando explicitamente variavel com mes da bilhetagem sem dicionario
del mes_d1_T

In [ ]:
# Base da Bilhetagem adicionado o dicionario
mes_d1

In [ ]:
mes_d1.info()

In [ ]:
# Criando df somente colunas necessátrias para a integração no tópico 03!
mes_geo = mes_d1.sort_values('momento')

In [ ]:
# Reiniciando a contagem dos index
mes_geo.reset_index(inplace=True)

In [ ]:
# REMOVENDO COLUNA DESNECESSÁRIA!
mes_georr = mes_geo.drop(['index', "prefixo_carro"], axis=1)

In [ ]:
mes_georr.info()

In [ ]:
mes_georr

In [ ]:
# Deletando explicitamente variavel com mes da bilhetagem sem dicionario
del mes
del mes_d1
del mes_geo

# 3.0 Integração das coordenadas do ônibus na bilhetagem e estimação do possivel local de validação
''' Método: Assumindo velocidade constante entre dois registros do GPS para determinação do local de validação da bilhetagem mais provável nesse trecho '''

## 3.1 Adicioando informações geometricas  pelo momento (GPS) na base da bilhetagem
''' Trazendo latitude e longitude para a base da bilhetagem pela momento anterior e posterior do GPS, isso para um mesmo veiculo '''

In [ ]:
# Criando função para identificar o momento anterior e posterior do GPS ao momento da bilhetagem, para um mesmo veiculo
def identificação_momentos(mes_georr, GPS_I_H):
    
    # Encontrando o momento anterior na bilhetagem
    df2_anterior = pd.merge_asof(mes_georr[['momento', 'id_veiculo']], GPS_I_H[['momento_gps', 'id_veiculo', 'longitude', 'latitude']], left_on='momento', by='id_veiculo', right_on='momento_gps', direction='backward')

    # Encontrando o momento posterior na bilhetagem
    df2_proximo = pd.merge_asof(mes_georr[['momento', 'id_veiculo']], GPS_I_H[['momento_gps', 'id_veiculo', 'longitude', 'latitude']], left_on='momento',  by='id_veiculo', right_on='momento_gps', direction='forward')

    # Atribuindo valores para o momento anterior, longitude e latitude
    if not df2_anterior.empty:
        mes_georr['momento_anterior_gps'] = df2_anterior['momento_gps'].values
        mes_georr['longitude_anterior_gps'] = df2_anterior['longitude'].values
        mes_georr['latitude_anterior_gps'] = df2_anterior['latitude'].values
    else:
        mes_georr['momento_anterior_gps'] = pd.NaT
        mes_georr['longitude_anterior_gps'] = pd.NaT
        mes_georr['latitude_anterior_gps'] = pd.NaT

    # Atribuindo valores para o momento posterior, longitude e latitude
    if not df2_proximo.empty:
        mes_georr['momento_posterior_gps'] = df2_proximo['momento_gps'].values
        mes_georr['longitude_posterior_gps'] = df2_proximo['longitude'].values
        mes_georr['latitude_posterior_gps'] = df2_proximo['latitude'].values
    else:
        mes_georr['momento_posterior_gps'] = pd.NaT
        mes_georr['longitude_posterior_gps'] = pd.NaT
        mes_georr['latitude_posterior_gps'] = pd.NaT

    return mes_georr

In [ ]:
# Aplicando a função
mes_momentos_geometry = identificação_momentos(mes_georr, dia_especifico_gps_bruto_)

In [ ]:
# Geometricas com dados nulos, ou seja, não se teve um momento anterior ou posterior para associar 
longitude_anterior_gps = mes_momentos_geometry['longitude_anterior_gps'].isnull().sum()
print("Dados nulos da coluna: longitude_anterior_gps -", longitude_anterior_gps)
longitude_posterior_gps = mes_momentos_geometry['longitude_posterior_gps'].isnull().sum()
print("Dados nulos da coluna: longitude_posterior_gps -", longitude_posterior_gps)

## 3.2 Estimando local de validação
''' Ja se tendo o momento anterior, posterior e dados geometricos da validação é possivel relacionar no trecho, adimitindo velocidade constante e 
determinar o possivel local de validação '''

### 3.2.1 Determinação da distancia estimada entre local de validação e registro imediatamente anterior do GPS

In [ ]:
# importando os objetos geométricos da biblioteca shapely e importando geopandas
!pip install -q shapely
!pip install -q geopandas
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [ ]:
# Funçao para criar uma so coluna de informação geometrica para dados de localização anterior e posterior a bilhetagem
def criar_ponto_anterior(row):
    if pd.notnull(row['longitude_anterior_gps']) and pd.notnull(row['latitude_anterior_gps']):
        return Point(row['longitude_anterior_gps'], row['latitude_anterior_gps'])
    else:
        return None
    
def criar_ponto_posterior(row):
    if pd.notnull(row['longitude_posterior_gps']) and pd.notnull(row['latitude_posterior_gps']):
        return Point(row['longitude_posterior_gps'], row['latitude_posterior_gps'])
    else:
        return None

In [ ]:
print("\n", ''' Essas funções para retornarem None nos dados de longitude e latitude nulos é devido não ser possivel a criação diretamente de pontos geometricos nulos com a função Point ''', "\n") 

In [ ]:
# Aplicando funções para criar geometrias 
mes_momentos_geometry['geometry_anterior_gps'] = mes_momentos_geometry.apply(criar_ponto_anterior, axis=1)
mes_momentos_geometry['geometry_posterior_gps'] = mes_momentos_geometry.apply(criar_ponto_posterior, axis=1)

In [ ]:
# Eliminando colunas desnecessárias 
mes_geometry_ = mes_momentos_geometry.drop(['longitude_anterior_gps', "latitude_anterior_gps", "longitude_posterior_gps", "latitude_posterior_gps"], axis=1)

In [ ]:
# Substituindo geometrias nulas pela coordenada 0,0  ## Dados nulos foram deixados!
# mes_geometry_['geometry_anterior'].fillna(Point(0,0), inplace= True)
# mes_geometry_['geometry_posterior'].fillna(Point(0,0), inplace= True)

In [ ]:
# Deletando explicitamente variavel com mes da bilhetagem 
del mes_momentos_geometry

In [ ]:
# Criando um GeoDataFrame
mes_geometry = gpd.GeoDataFrame(mes_geometry_, geometry='geometry_anterior_gps')
mes_geometry = gpd.GeoDataFrame(mes_geometry_, geometry='geometry_posterior_gps')

In [ ]:
# Deletando explicitamente variavel com mes da bilhetagem 
del mes_geometry_

In [ ]:
# Importando modulo e biblioteca para calculo de distancia entre pontos geometricos
!pip install geopy
from geopy.distance import geodesic 

In [ ]:
# Criando função para calcular distancia entre dois pontos geometericos
def calcular_distancia_gps(row):
    posterior_geometry_gps = row['geometry_posterior_gps']
    anterior_geometry_gps = row['geometry_anterior_gps']
    
    if posterior_geometry_gps is None or anterior_geometry_gps is None:
        return None
    return geodesic((posterior_geometry_gps.x, posterior_geometry_gps.y), (anterior_geometry_gps.x, anterior_geometry_gps.y)).meters

In [ ]:
# Criando coluna de distancia entre cada par de registros do GPS
mes_geometry['distancia_gps'] = mes_geometry.apply(calcular_distancia_gps, axis=1)

In [ ]:
# Calculando diferença de tempos
mes_geometry['deltaT_gps'] = mes_geometry['momento_posterior_gps'] - mes_geometry['momento_anterior_gps']

In [ ]:
print("\n", '''A coluna "deltaT_gps" é a diferenca do tempo de registro para cada trecho do GPS que cada validação da Bilhetagem se encontra''', "\n") 

In [ ]:
# Criando coluna de diferença entre o tempo da validação e o tempo do registro do GPS imediatamente anterior
mes_geometry['deltaT_validação_gps'] = (mes_geometry['momento'] - mes_geometry['momento_anterior_gps'])

In [ ]:
# Transformando colunas de deltatT em segundos 
mes_geometry['deltaT_gps'] = mes_geometry['deltaT_gps'].dt.total_seconds().astype('float')
mes_geometry['deltaT_validação_gps'] = mes_geometry['deltaT_validação_gps'].dt.total_seconds().astype('float')

In [ ]:
# Remover colunas agora desnecessárias 
mes_geometry_dist = mes_geometry.drop(['momento_anterior_gps'], axis=1)

In [ ]:
# Deletando explicitamente variavel com mes da bilhetagem 
del mes_geometry

In [ ]:
# REALIZANDO CALCULO DA DISTANCIA ESTIMADA PARA CADA VALIDAÇÃO EM RELAÇÃO AO REGISTRO DO GPS IMEDIATAMENTE ANTERIOR
mes_geometry_dist['distancia_validação_gps'] = (mes_geometry_dist['distancia_gps']/mes_geometry_dist['deltaT_gps'])* mes_geometry_dist['deltaT_validação_gps']

In [ ]:
# Remover colunas agora desnecessárias 
mes_geometry_dist = mes_geometry_dist.drop(['deltaT_gps', 'deltaT_validação_gps'], axis=1)

In [ ]:
# Df com distancias entre a validação e o registro do GPS anterior e a coordenada desse registro 
mes_geometry_dist

In [1]:
print("\n", '''Ou seja, a operação faz a mutiplicação do cociente entre a diferenca de distancia no trecho e do tempo pela variação do tempo entre a validação e o registro do GPS anterior, o que é basicamente a multiplicação em cada trecho da velocidade pela variação no tempo da validação e o registro do ponto anteiror do GPS (adimitindo que nesse trecho a velocidade seja constante, isto é, a mesma do trecho total)''', "\n")


 Ou seja, a operação faz a mutiplicação do cociente entre a diferenca de distancia no trecho e do tempo pela variação do tempo entre a validação e o registro do GPS anterior, o que é basicamente a multiplicação em cada trecho da velocidade pela variação no tempo da validação e o registro do ponto anteiror do GPS (adimitindo que nesse trecho a velocidade seja constante, isto é, a mesma do trecho total) 



#### 3.2.1.1 Avaliando dados das distancias das validações em relação ao  registro anterior mais proximo do gps

In [ ]:
# Avaliando distancias 
import numpy as np
import matplotlib.pyplot as plt

# Criando variavel da distancia sem dados nulos
dist_sem_nulos = mes_geometry_dist["distancia_validação_gps"].dropna()

In [ ]:
# Identificando tamnhos dessas distancias e se fazem sentido para o contexto
plt.boxplot(dist_sem_nulos)
plt.ylabel("distância (m)")

In [2]:
print("\n", ''' Ou seja, devido intervalos muito grande entre a contagem dos registros do GPS, ocasiona distancias muito grande, mas elas não fazem parte do objeto de estudo do metodo aplicado ''', "\n") 


  Ou seja, devido intervalos muito grande entre a contagem dos registros do GPS, ocasiona distancias muito grande, mas elas não fazem parte do objeto de estudo do metodo aplicado  



In [ ]:
# Quartis 
Qurtil_Q1 = np.percentile(dist_sem_nulos, 25)
Qurtil_Q2 = np.percentile(dist_sem_nulos, 50)
Qurtil_Q3 = np.percentile(dist_sem_nulos, 75)
print("\n", "Primeiro quartil é:", Qurtil_Q1,"\n", "Segundo quartil é:", Qurtil_Q2,"\n", "Terceiro quartil é:", Qurtil_Q3, "\n")

In [3]:
print("\n", ''' Assim identificamos que 75% dos valores possuem distancias de até "valor do Terceiro quartil (metros)", então vamos adotar um criterio de que para analisar melhor as distancias, so faz sentido olhar para aquelas que seja de até 500 metros, supondo que onibus esteja no maximo a 60 km/h para cumprir um intervalo maximo de 30 segundos, ou seja, a essa velocidade esse seria o limite supeior de distancia que o local de validaçao poderia estar, que é exatamente a proxima localização do registro do GPS se ele contar efetivamente a cada 30 segundos ''', "\n") 


  Assim identificamos que 75% dos valores possuem distancias de até "valor do Terceiro quartil (metros)", então vamos adotar um criterio de que para analisar melhor as distancias, so faz sentido olhar para aquelas que seja de até 500 metros, supondo que onibus esteja no maximo a 60 km/h para cumprir um intervalo maximo de 30 segundos, ou seja, a essa velocidade esse seria o limite supeior de distancia que o local de validaçao poderia estar, que é exatamente a proxima localização do registro do GPS se ele contar efetivamente a cada 30 segundos  



In [ ]:
# Filtrando valores de distancias a partir desse criterio
dist_ate_500_metros = dist_sem_nulos[dist_sem_nulos<= 500]

In [ ]:
# Nova visulização desses valores 
plt.boxplot(dist_ate_500_metros)
plt.ylabel("distância (m)")

In [ ]:
# Média das distancias
média = dist_ate_500_metros.mean()
print("A média das distancias é de:", média, "(m)")

In [ ]:
# E 90% dos dados?
Qurtil_Q_90 = np.percentile(dist_ate_500_metros, 90)
print("90% das distancias entre validação e primerio registro do gps de até:", Qurtil_Q_90, "metros")

In [ ]:
# Desvio padrão 
desvio = dist_ate_500_metros.std()
desvio

In [ ]:
# Coeficiente de variação
coef = desvio/média *100
print("coeficinte de variação é de: ", coef, "%")

In [ ]:
# Quartis da base sem grandes distancais
Qurtil_Q1 = np.percentile(dist_ate_500_metros, 25)
Qurtil_Q2 = np.percentile(dist_ate_500_metros, 50)
Qurtil_Q3 = np.percentile(dist_ate_500_metros, 75)
print("\n", "Primeiro quartil é:", Qurtil_Q1,"\n", "Segundo quartil é:", Qurtil_Q2,"\n", "Terceiro quartil é:", Qurtil_Q3, "\n")

In [4]:
print("\n", ''' Percebemos que ha uma leve mudanca nos valores dos quartis, isso pode ser devido essas quandes distancias serem um valor pequeno em comparação as demais distancias ''', "\n")


  Percebemos que ha uma leve mudanca nos valores dos quartis, isso pode ser devido essas quandes distancias serem um valor pequeno em comparação as demais distancias  



In [ ]:
# Maxima distancia 
dist_ate_500_metros.max()

#### 3.2.1.2 Avaliando dados das distancias entres os registros anteriores e posteriores mais proximo das validações

In [ ]:
# Criando variavel da distancia sem dados nulos
dist_gps_sem_nulos = mes_geometry_dist["distancia_gps"].dropna()

In [ ]:
# Identificando tamnhos dessas distancias 
plt.boxplot(dist_gps_sem_nulos)
plt.ylabel("distância (m)")

In [ ]:
# Quartis 
Qurtil_Q1_gps = np.percentile(dist_gps_sem_nulos, 25)
Qurtil_Q2_gps = np.percentile(dist_gps_sem_nulos, 50)
Qurtil_Q3_gps = np.percentile(dist_gps_sem_nulos, 75)
print("\n", "Primeiro quartil é:", Qurtil_Q1_gps,"\n", "Segundo quartil é:", Qurtil_Q2_gps,"\n", "Terceiro quartil é:", Qurtil_Q3_gps, "\n")

In [ ]:
# E 90% dos dados?
Qurtil_Q_90_gps = np.percentile(dist_gps_sem_nulos, 90)
print("90% das distancias entre os registros apresetam valores até:", Qurtil_Q_90_gps, "metros")

In [ ]:
# Média das distancias entre os registros do gps
média_gps = dist_gps_sem_nulos.mean()
print("A média das distancias entre os registros do gps é de:", média_gps, "(m)")

In [ ]:
# Desvio padrão 
desvio_gps = dist_gps_sem_nulos.std()
desvio_gps

### 3.2.2 Realizando operação de adição da distancia estimada ao registro anterior do GPS
''' Ja se tendo a distancia estimada entre a validação e o registro do GPS imediatamente anterior, basta adcionar ela a
essa coordenada e a nova coordenada é exatamente a possivel coordenada onde ocorreu a validação '''

In [ ]:
# Instalando biblioteca para adicionar as distancias aos seus repectivos pontos geometricos
!pip install -q pyproj
!pip install -q shapely

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import transform
from functools import partial
import math
import pyproj
! pip install --upgrade pyproj shapely

In [ ]:
# Atualizando sitaxe de projeção 
import pyproj

# Sintaxe obsoleta
old_projection = pyproj.Proj(init='epsg:4326')

# Atualização para a nova sintaxe
new_projection = pyproj.Proj('epsg:4326')

In [ ]:
# Criando função para calcular o ângulo entre dois pontos geometricos
def calcular_angulo(ponto_inicial, ponto_final):
    # Verifica se ambos os pontos são não nulos
    if ponto_inicial is None or ponto_final is None:
        return None
    
    # Calcula as diferenças nas coordenadas x e y
    delta_x = ponto_final.x - ponto_inicial.x
    delta_y = ponto_final.y - ponto_inicial.y

    # Calcula o ângulo em radianos usando a função atan2
    angulo_rad = math.atan2(delta_y, delta_x)

    # Converte o ângulo para graus
    angulo_graus = math.degrees(angulo_rad)

    return angulo_graus

In [ ]:
# Aplicando função nas duas colunas geometricas
mes_geometry_dist['angulo'] = mes_geometry_dist.apply(lambda row: calcular_angulo(row['geometry_anterior_gps'], row['geometry_posterior_gps']), axis=1)

In [ ]:
# Df com angulo
mes_geometry_dist

In [ ]:
# Criando função para criar um ponto deslocado
def offset_distancia(row):
    # Função para calcular as coordenadas do novo ponto deslocado
    def calcular_coordenadas(geometry_anterior_gps, distancia_validação_gps, angulo):
        # Verificando se o ponto não é None
        if geometry_anterior_gps is None:
            return None

        # Convertendo a distância de metros para graus (aproximadamente)
        distancia_graus_latitude = (distancia_validação_gps / 111000)
        distancia_graus_longitude = (distancia_validação_gps / (111000 * math.cos(math.radians(geometry_anterior_gps.y))))

        # Calculando as novas coordenadas
        new_latitude = geometry_anterior_gps.y + distancia_graus_latitude * math.sin(math.radians(angulo))
        new_longitude = geometry_anterior_gps.x + distancia_graus_longitude * math.cos(math.radians(angulo))

        return Point(new_longitude, new_latitude)

    # Aplica a função para calcular as novas coordenadas a cada linha do DataFrame
    new_point = calcular_coordenadas(row['geometry_anterior_gps'], row['distancia_validação_gps'], row['angulo'])

    return new_point

In [ ]:
# Aplicando a função ao DataFrame
mes_geometry_dist['geometry_estimada'] = mes_geometry_dist.apply(offset_distancia, axis=1)

In [ ]:
# Bilhetagem com local de validação estimado
mes_geometry_dist

In [ ]:
# Caso necessite exportar 
# mes_geometry_dist.to_csv("NOME_ARQUIVO")

# 4.0 Testes

## 4.1 Motivo do não georreferenciamento da base completa da bilhetagem

In [ ]:
# Inspeção na base da bilhetagem e do dicionario para um veiculo especifico
veiculo_12994 = mes[mes.prefixo_carro==12994]

In [ ]:
# NOTAMOS QUE ESSE VEICULO ESPECIFICO POSSUI REGISTROS NA BILHETAGEM

In [ ]:
### Verificando dados que estão presentes na base da bilhetagem e não na base do dicionario
lista = d01.cod_veiculo.unique()
mes.isin({'prefixo_carro':lista}).prefixo_carro.value_counts(1)

In [ ]:
# OU SEJA, DOS 100% DE DADOS NA BILHETAGEM APENAS 86,78% POSSUI UM CODIGO CORRESPONDENTE NA BASE DO DICIONARIO!

In [ ]:
# VERIFICAÇÃO DO VEICULO 12994 NA BASE DO DICIONARIO 
d01[d01.cod_veiculo == 12993]

In [5]:
''' CONCLUSÃO, APÓS INSPEÇÃO NOTAMOS QUE NÃO FOI POSSIVEL GEORREFERENCIAR A BASE DA BILHETAGEM COMPLETA POIS PARA ALGUNS VEICULOS NÃO TEM UM CODIGO DE VEICULO CORRESPONDENTE NA BASE DO DICIONARIO E CONSEQUENTIMENTE NÃO VAI SER LOCALIZADO NA BASE DO GPS, ASSIM UMA PERCENTAGEM DOS DADOS DA BILHETAGEM TIVERAM COORDENADA APROXIMADA DE VALIDAÇÃO GEORREFERENCIADA e o metodo aplicado não considerou os dados com caracters do dicionario, uma abordagem tratando esses dados sem removelos pode aumentar a quantidade de validações localizadas! ''' 

' CONCLUSÃO, APÓS INSPEÇÃO NOTAMOS QUE NÃO FOI POSSIVEL GEORREFERENCIAR A BASE DA BILHETAGEM COMPLETA POIS PARA ALGUNS VEICULOS NÃO TEM UM CODIGO DE VEICULO CORRESPONDENTE NA BASE DO DICIONARIO E CONSEQUENTIMENTE NÃO VAI SER LOCALIZADO NA BASE DO GPS, ASSIM UMA PERCENTAGEM DOS DADOS DA BILHETAGEM TIVERAM COORDENADA APROXIMADA DE VALIDAÇÃO GEORREFERENCIADA e o metodo aplicado não considerou os dados com caracters do dicionario, uma abordagem tratando esses dados sem removelos pode aumentar a quantidade de validações localizadas! '

# -------------------------------------------------------------------